In [1]:
import os
import sys

In [2]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed,wait,fire_and_forget,progress
import glob
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import importlib
import dask
import zarr
import time
from rechunker import rechunk

In [3]:
def custom_optimize(dsk, keys):
    dsk = dask.optimization.inline(dsk, inline_constants=True)
    return dask.array.optimization.optimize(dsk, keys)
sns.set_theme()
dask.config.set({"distributed.comm.timeouts.tcp": "50s"})
dask.config.set({'distributed.comm.retry.count':3})
dask.config.set({'distributed.comm.timeouts.connect':'25s'})
dask.config.set({"distributed.worker.use-file-locking":False})
zarr.blosc.use_threads = False

In [4]:
batches=5

In [5]:
cluster=SLURMCluster(cores=8,
                     memory="50GB",
                     queue='q36',
                     walltime='0-06:00:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     scheduler_options={'interface':'ib0'},
                     extra=['--resources mem=15',"--lifetime", "340m", "--lifetime-stagger", "4m"]
                    )
                                

In [6]:
client=Client(cluster)

In [7]:
cluster.adapt(minimum_jobs=1,maximum_jobs=4)

In [8]:
client

Client Scheduler: tcp://10.100.6.3:32907 Dashboard: http://10.100.6.3:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
data='../../data/interim/data.zarr/'
ds=xr.open_zarr(data)
#ds=xr.open_mfdataset("/home/au643300/NOBACKUP/interim/*nc",parallel=True,combine='nested',concat_dim='time',chunks='auto')

In [5]:
ds

,Array,Chunk
Bytes,66.84 GiB,100.50 MiB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,66.84 GiB,100.50 MiB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


Constants from the simulation

In [11]:
Re_Tau = 395 #Direct from simulation
Re = 10400 #Direct from simulation
nu = 1/Re #Kinematic viscosity
u_tau = Re_Tau*nu #The friction velocity 
#y_plus=(y*utau/nu)


changes y til y+. goes from 790 to 0. so y_plus[0]=790

In [12]:
ds=ds.assign_coords(y=(ds.y*u_tau/nu))
ds=ds.rename({'y':'y_plus'})
ds

,Array,Chunk
Bytes,66.84 GiB,100.50 MiB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,66.84 GiB,100.50 MiB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


First the computations are made for the velocities

In [13]:
ds

,Array,Chunk
Bytes,66.84 GiB,100.50 MiB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,66.84 GiB,100.50 MiB
Shape,"(681, 256, 201, 256)","(1, 256, 201, 256)"
Count,682 Tasks,681 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


Kører over alle de forskellige variable i datasettet. Skifter mellem skalaerer ved k=3

In [14]:
def calc_stats(dataset,u_tau):
    ds=dataset
    val_list=list(ds.keys())
    val_list=sorted(val_list)
    mean=ds
    mean=mean.drop(labels=val_list)
    k=0
    for val in val_list:
        mean_tem=ds[val]/u_tau
        mean_tem=xr.DataArray.mean(mean_tem,dim=('x','z'))
        rms_tem=(((ds[val]/u_tau)-mean_tem)**2)
        rms_tem=xr.ufuncs.sqrt(xr.DataArray.mean(rms_tem,dim=('x','z')))
        mean[val[0]+'_plusmean']=mean_tem
        mean[val[0]+'_plusRMS']=rms_tem
        if k<4: #So for the Pr numbers
            mean=mean.drop(labels=val[0]+'_plusmean')
            mean=mean.drop(labels=val[0]+'_plusRMS')
            pr=float(val[2:])
            theta_tau=(mean_tem.isel(y_plus=1)-mean_tem.isel(y_plus=0))/((mean_tem.y_plus[0]-mean_tem.y_plus[1])*pr)
            theta_plus=mean_tem/theta_tau
            rms_pr=rms_tem/theta_tau
            mean[val+'_plusmean']=theta_plus
            mean[val+'_plusRMS']=rms_pr
        k=k+1
    return mean

In [15]:
mean=calc_stats(ds,u_tau)

In [16]:
# %%time
# mean.to_zarr('data/interim/stats.nc')

In [17]:
# %%time
# mean.to_netcdf('data/interim/stats.nc')

In [19]:
with dask.config.set(array_optimize=custom_optimize):
    mean=calc_stats(ds,u_tau)
    mean=dask.optimize(mean)[0]
    mean.to_netcdf('../../data/interim/stats.nc')